# Overview
- 間接相互作用の距離を計算する

# Import everyting I need :)

In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from plotly.graph_objs import Scatter3d
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from tqdm import tqdm
# from fastprogress import progress_bar
import warnings
warnings.filterwarnings('ignore')
import multiprocessing as multi
from multiprocessing import Pool

# Preparation

In [125]:
isSmallSet = False
length = 20000
file_path = '../input/champs-scalar-coupling/'
nb = 33

In [126]:
pd.set_option('display.max_columns', 100)

In [127]:
# train
path = file_path + 'train.csv'
if isSmallSet:
    train = pd.read_csv(path)[:length]
else:
    train = pd.read_csv(path)

In [128]:
# test
path = file_path + 'test.csv'
if isSmallSet:
    test = pd.read_csv(path)[:length]
else:
    test = pd.read_csv(path)

In [129]:
# structure
path = file_path + 'structures.csv'
structures = pd.read_csv(path)

# create features

In [130]:
# compute bonds
atomic_radius = {'H':0.38, 'C':0.77, 'N':0.75, 'O':0.73, 'F':0.71} # Without fudge factor
fudge_factor = 0.05
atomic_radius = {k:v + fudge_factor for k,v in atomic_radius.items()}

atoms = structures['atom'].values
atoms_rad =[atomic_radius[x] for x in atoms]
structures['rad'] = atoms_rad

i_atom = structures['atom_index'].values
p = structures[['x', 'y', 'z']].values
p_compare = p
m = structures['molecule_name'].values
m_compare = m
r = structures['rad'].values
r_compare = r

source_row = np.arange(len(structures))
max_atoms = 28

bonds = np.zeros((len(structures)+1, max_atoms+1), dtype=np.int8)
bond_dists = np.zeros((len(structures)+1, max_atoms+1), dtype=np.float32)

print('Calculating bonds')

for i in tqdm(range(max_atoms-1)):
    p_compare = np.roll(p_compare, -1, axis=0)
    m_compare = np.roll(m_compare, -1, axis=0)
    r_compare = np.roll(r_compare, -1, axis=0)
    
    mask = np.where(m == m_compare, 1, 0) #Are we still comparing atoms in the same molecule?
    dists = np.linalg.norm(p - p_compare, axis=1) * mask
    r_bond = r + r_compare
    
    bond = np.where(np.logical_and(dists > 0.0001, dists < r_bond), 1, 0)
    
    source_row = source_row
    target_row = source_row + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_row = np.where(np.logical_or(target_row > len(structures), mask==0), len(structures), target_row) #If invalid target, write to dummy row
    
    source_atom = i_atom
    target_atom = i_atom + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_atom = np.where(np.logical_or(target_atom > max_atoms, mask==0), max_atoms, target_atom) #If invalid target, write to dummy col
    
    bonds[(source_row, target_atom)] = bond
    bonds[(target_row, source_atom)] = bond
    bond_dists[(source_row, target_atom)] = dists
    bond_dists[(target_row, source_atom)] = dists

bonds = np.delete(bonds, axis=0, obj=-1) #Delete dummy row
bonds = np.delete(bonds, axis=1, obj=-1) #Delete dummy col
bond_dists = np.delete(bond_dists, axis=0, obj=-1) #Delete dummy row
bond_dists = np.delete(bond_dists, axis=1, obj=-1) #Delete dummy col

print('Counting and condensing bonds')

bonds_numeric = [[i for i,x in enumerate(row) if x] for row in tqdm(bonds)]
bond_lengths = [[dist for i,dist in enumerate(row) if i in bonds_numeric[j]] for j,row in enumerate(tqdm(bond_dists))]
n_bonds = [len(x) for x in bonds_numeric]

#bond_data = {'bond_' + str(i):col for i, col in enumerate(np.transpose(bonds))}
#bond_data.update({'bonds_numeric':bonds_numeric, 'n_bonds':n_bonds})

bond_data = {'bonds':bonds_numeric, 'n_bonds':n_bonds, 'bond_lengths':bond_lengths}
bond_df = pd.DataFrame(bond_data)
structures = structures.join(bond_df)
display(structures.head(20))

  0%|          | 0/27 [00:00<?, ?it/s]

Calculating bonds


  1%|          | 25613/2358657 [00:00<00:09, 256123.53it/s]

Counting and condensing bonds


100%|██████████| 2358657/2358657 [00:13<00:00, 180693.49it/s]


,molecule_name,atom_index,atom,x,y,z,rad,bonds,n_bonds,bond_lengths
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,0.82,"[1, 2, 3, 4]",4,"[1.091953, 1.0919516, 1.0919464, 1.0919476]"
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,0.43,[0],1,[1.091953]
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,0.43,[0],1,[1.0919516]
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,0.43,[0],1,[1.0919464]
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,0.43,[0],1,[1.0919476]
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564,0.80,"[1, 2, 3]",3,"[1.01719, 1.0171872, 1.0172079]"
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377,0.43,[0],1,[1.01719]
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758,0.43,[0],1,[1.0171872]
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543,0.43,[0],1,[1.0172079]
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602,0.78,"[1, 2]",2,"[0.9621068, 0.9621068]"


In [131]:
def map_atom_info(df_1,df_2, atom_idx):
    df = pd.merge(df_1, df_2, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    df = df.drop('atom_index', axis=1)

    return df


for atom_idx in [0,1]:
    train = map_atom_info(train, structures, atom_idx)
    test  = map_atom_info(test, structures, atom_idx)
    
    train = train.rename(columns={'atom': f'atom_{atom_idx}',
                                        'x': f'x_{atom_idx}',
                                        'y': f'y_{atom_idx}',
                                        'z': f'z_{atom_idx}',
                                        'rad': f'rad_{atom_idx}',
                                        'bonds': f'bonds_{atom_idx}',
                                        'n_bonds': f'n_bonds_{atom_idx}',
                                        'bond_lengths': f'bond_lengths_{atom_idx}'})
    test  =  test.rename(columns={'atom': f'atom_{atom_idx}',
                                        'x': f'x_{atom_idx}',
                                        'y': f'y_{atom_idx}',
                                        'z': f'z_{atom_idx}',
                                        'rad': f'rad_{atom_idx}',
                                        'bonds': f'bonds_{atom_idx}',
                                        'n_bonds': f'n_bonds_{atom_idx}',
                                        'bond_lengths': f'bond_lengths_{atom_idx}'})

In [132]:
def distances(df):
    df_p_0 = df[['x_0', 'y_0', 'z_0']].values
    df_p_1 = df[['x_1', 'y_1', 'z_1']].values
    
    df['dist'] = np.linalg.norm(df_p_0 - df_p_1, axis=1)
    df['dist_x'] = (df['x_0'] - df['x_1']) ** 2
    df['dist_y'] = (df['y_0'] - df['y_1']) ** 2
    df['dist_z'] = (df['z_0'] - df['z_1']) ** 2
    
    return df

train = distances(train)
test  = distances(test)

-----

---

In [133]:
def compute_dist(idx0, idx1, structure):
    idx = structure['atom_index']==idx0
    p_0 = structure[['x', 'y', 'z']].values[idx]
    idx = structure['atom_index']==idx1
    p_1 = structure[['x', 'y', 'z']].values[idx]
    return np.linalg.norm(p_0 - p_1, axis=1)[0]
    
def compute_interact_dist(data, mole_name = 'dsgdb9nsd_000001'):
    mole = data[data['molecule_name']==mole_name]
    struct = structures[structures['molecule_name']==mole_name]
    
    sum_dists = []
    for i_type, type_ in enumerate(mole['type']):
#             print(mole.id)
            type_num = type_.split('J')[0]
            mole_row = mole.iloc[i_type,:]
            atom_idx = mole_row['atom_index_0']
            interact_idx = mole_row['atom_index_1']

            if type_num == '1':
                dist_1 = compute_dist(atom_idx, interact_idx, struct)
                sum_dist = dist_1
                dists = [dist_1]

            # 1st bond
            bond_idx_1st = struct['bonds'].values[atom_idx][0]
            if type_num == '2':
                dist_1 = compute_dist(atom_idx, bond_idx_1st, struct)
                dist_2 = compute_dist(bond_idx_1st, interact_idx, struct)
                sum_dist = dist_1 + dist_2
                dists = [dist_1, dist_2]

            # 2nd bond list
            bond_idx_2nd_list = struct['bonds'].values[bond_idx_1st]
            bond_atom_2nd_list = struct['atom'].iloc[bond_idx_2nd_list].values.tolist()
            if type_num == '3':
                logi = []
                for i, atom in enumerate(bond_atom_2nd_list):
                    logi.append(atom in ['C', 'O', 'N'])
                _bond_atom_2nd_list = np.array(bond_atom_2nd_list)[logi].tolist()
                _bond_idx_2nd_list = np.array(bond_idx_2nd_list)[logi].tolist()

                # 2nd bond list の中に、相互作用先の原子がいるはず
                for i in _bond_idx_2nd_list:
                    bonds = struct['bonds'].values[i]
                    if interact_idx in bonds :
                        bond_idx_2nd = i
                
                try:
                    dist_1 = compute_dist(atom_idx, bond_idx_1st, struct)
                    dist_2 = compute_dist(bond_idx_1st, bond_idx_2nd, struct)
                    dist_3 = compute_dist(bond_idx_2nd, interact_idx, struct)
                    sum_dist = dist_1 + dist_2 + dist_3
                except:
                    sum_dist = np.nan
#                     print(bonds)
#                     print(interact_idx)
#                 dists = [dist_1, dist_2, dist_3]
            sum_dists.append(sum_dist)
    return sum_dists
# sum_dists = compute_interact_dist(mole_name = 'dsgdb9nsd_000002')

- 相互作用距離の計算

In [134]:
%%time
# train
# マルチプロセス
# train['dist_interact'] = 0.0
def process(i):
#     global train
    mole_name = train['molecule_name'].unique()[i]
    sum_dists = compute_interact_dist(train, mole_name)
    idx = train['molecule_name']==mole_name
#     train['dist_interact'][idx] = np.array(sum_dists)
#     df = pd.DataFrame(train[['id', 'dist_interact']][idx])
    df = pd.DataFrame({'dist_interact': np.array(sum_dists)})
    return df
    
#     train['dist_interact'] = 1
    
n_job = 75
n_molecule = len(train['molecule_name'].unique())

with Pool(n_job) as pool:
    r = range(n_molecule)
    imap = pool.imap(process, r)
    result = list(tqdm(imap, total=len(r)))
    train['dist_interact'] = pd.concat(result).values

100%|██████████| 85003/85003 [46:32<00:00, 30.44it/s]


CPU times: user 1min 29s, sys: 25.5 s, total: 1min 54s
Wall time: 46min 47s


In [135]:
%%time
# test
# マルチプロセス
# test['dist_interact'] = 0.0
def process(i):
#     global test
    mole_name = test['molecule_name'].unique()[i]
    sum_dists = compute_interact_dist(test, mole_name)
    idx = test['molecule_name']==mole_name
#     test['dist_interact'][idx] = np.array(sum_dists)
#     df = pd.DataFrame(test[['id', 'dist_interact']][idx])
    df = pd.DataFrame({'dist_interact': np.array(sum_dists)})
    return df
    
#     test['dist_interact'] = 1
    
n_job = 75
n_molecule = len(test['molecule_name'].unique())

with Pool(n_job) as pool:
    r = range(n_molecule)
    imap = pool.imap(process, r)
    result = list(tqdm(imap, total=len(r)))
    test['dist_interact'] = pd.concat(result).values

100%|██████████| 45772/45772 [18:47<00:00, 40.59it/s]


CPU times: user 46.3 s, sys: 16 s, total: 1min 2s
Wall time: 19min


# Save Feature

In [145]:
# train
path = file_path + f'nb{nb}_train_dist-interaction.csv'
df = pd.DataFrame({'dist_interact': train['dist_interact'].values})
# df.to_csv(path, index=False)

In [137]:
# test
path = file_path + f'nb{nb}_test_dist-interaction.csv'
df = pd.DataFrame({'dist_interact': test['dist_interact'].values})
df.to_csv(path, index=False)

In [146]:
df.head()

,dist_interact
0,1.091953
1,2.183905
2,2.183899
3,2.183901
4,1.091952


# EDA

In [144]:
name = train['molecule_name'].unique()[0]
idx = train['molecule_name']==name
sum_dists = compute_interact_dist(train, name)
display(train[['type', 'molecule_name', 'atom_index_0', 'atom_index_1', 'dist', 'dist_interact']][idx])
# plot_atom(name)

,type,molecule_name,atom_index_0,atom_index_1,dist,dist_interact
0,1JHC,dsgdb9nsd_000001,1,0,1.091953,1.091953
1,2JHH,dsgdb9nsd_000001,1,2,1.783120,2.183905
2,2JHH,dsgdb9nsd_000001,1,3,1.783147,2.183899
3,2JHH,dsgdb9nsd_000001,1,4,1.783157,2.183901
4,1JHC,dsgdb9nsd_000001,2,0,1.091952,1.091952
5,2JHH,dsgdb9nsd_000001,2,3,1.783158,2.183898
6,2JHH,dsgdb9nsd_000001,2,4,1.783148,2.183899
7,1JHC,dsgdb9nsd_000001,3,0,1.091946,1.091946
8,2JHH,dsgdb9nsd_000001,3,4,1.783148,2.183894
9,1JHC,dsgdb9nsd_000001,4,0,1.091948,1.091948


In [140]:
train['molecule_name'].unique()[5]
idx = train['molecule_name']==name

'dsgdb9nsd_000008'